# Task 1

In [1]:
def findFakeCoin(coins, start=0, end=None):
    if end is None:
        end = len(coins)
    
    # If there is only one coin, it is the fake coin
    if end - start == 1:
        return start + 1  # Add one to the index because indexing starts at 0
    
    # Divide the coins into three equal parts
    third = (end - start + 1) // 3
    
    # Calculate the sum of each group
    sum_first_group = sum(coins[start:start+third])
    sum_second_group = sum(coins[start+third:start+2*third])
    sum_third_group = sum(coins[start+2*third:end])
    
    if sum_first_group == sum_second_group:
        # If the first group and the second group are equal, the fake coin is in the third group
        return findFakeCoin(coins, start+2*third, end)
    elif sum_first_group == sum_third_group:
        # If the first group and the third group are equal, the fake coin is in the second group
        return findFakeCoin(coins, start+third, start+2*third)
    else:
        # Otherwise, it is in the first group
        return findFakeCoin(coins, start, start+third)

# Example input
coins = [10, 10, 10, 9, 10, 10, 10, 10, 10]
# Find the position of the fake coin
print(findFakeCoin(coins))


4


# Task 2

In [2]:
# The 'heapify' function is used to adjust the subtree with i as the root into a max heap
def heapify(arr, n, i, d):
    largest = i
    # Check all the child nodes
    for j in range(1, d+1):
        child = d * i + j
        # If the child node is larger than the parent node, then mark it
        if child < n and arr[child] > arr[largest]:
            largest = child
    # If the largest is not the root
    if largest != i:
        # Swap
        arr[i], arr[largest] = arr[largest], arr[i]
        # Heapify the root
        heapify(arr, n, largest, d)

# The 'build_max_heap' function is used to build a max heap from an array
def build_max_heap(arr, d):
    n = len(arr)
    # Build a max heap
    for i in range((n - 2) // d, -1, -1):
        heapify(arr, n, i, d)

# The 'dHeapSort' function is used to perform heap sort on an array
def dHeapSort(nums, d):
    n = len(nums)

    # Build max heap
    build_max_heap(nums, d)

    # One by one extract elements
    for i in range(n - 1, 0, -1):
        # Move current root to end
        nums[0], nums[i] = nums[i], nums[0]
        # Call max heapify on the reduced heap
        heapify(nums, i, 0, d)

    return nums

# example input
nums = [7, 6, 5, 9, 8]
d = 3
print(dHeapSort(nums, d))  # Output: [5, 6, 7, 8, 9]

[5, 6, 7, 8, 9]


# Task 3

In [2]:
# solution
    # memo[node][remains] = min(memo[node-1][remains+dis]+charge(0),memo[node-1][remains+dis-1]+charge(1)....) --> memo[node+1][remains]
def calculate_charge_time(station, wait_times, charge_speeds, charge_amount):
    # If no charge is needed, return 0
    if charge_amount == 0:
        return 0
    # Otherwise, return the total time spent charging at this station
    return wait_times[station] + charge_speeds[station] * charge_amount

def timeSpent(distances, wait_times, charge_speeds):
    # Initialize car capacity and number of stations
    car_capacity = 400
    num_stations = len(distances)
    
    # Initialize memoization table for minimum time and last charge distance
    min_time = [[float('inf')] * (car_capacity + 1) for _ in range(num_stations)]
    last_charge_distance = [[0] * (car_capacity + 1) for _ in range(num_stations)]
    
    # The minimum time to reach the first station is 0
    min_time[0][0] = 0

    # Use dynamic programming to find the minimum time for each station
    for station in range(num_stations - 1):
        # Calculate the travel distance to the next station
        travel_distance = distances[station + 1] - distances[station]
        
        # If the travel distance is greater than the car's capacity, return -1
        if travel_distance > car_capacity:
            return -1
        
        # Calculate the remaining capacity after reaching the next station
        remaining_capacity = car_capacity - travel_distance if station != num_stations - 2 else 0
        
        # For each possible remaining capacity...
        for remains in range(remaining_capacity + 1):
            # Initialize the minimum time for this station and the corresponding charge distance
            min_time_for_station = float('inf')
            min_charge_distance = 0
            
            # For each possible charge distance...
            for charge_distance in range(remains + travel_distance + 1):
                # Calculate the time spent if charging this distance
                time = min_time[station][remains + travel_distance - charge_distance] + calculate_charge_time(station, wait_times, charge_speeds, charge_distance)
                
                # If this time is less than the current minimum time, update the minimum time and the corresponding charge distance
                if time < min_time_for_station:
                    min_time_for_station = time
                    min_charge_distance = charge_distance
            
            # Update the minimum time and the corresponding charge distance for this station and remaining capacity
            min_time[station + 1][remains] = min_time_for_station
            last_charge_distance[station + 1][remains] = min_charge_distance

    # Backtrack to find the time spent charging at each station
    remains = 0
    charge_time = []
    for i in range(num_stations - 1):
        # Calculate the station index
        station = num_stations - 1 - i
        
        # Calculate the travel distance to the next station
        travel_distance = distances[station] - distances[station - 1]
        
        # Find the charge distance that leads to the minimum time at this station
        charge_distance = last_charge_distance[station][remains]
        
        # Calculate the time spent charging this distance
        time = calculate_charge_time(station - 1, wait_times, charge_speeds, charge_distance)
        
        # Add this time to the beginning of the charge time list
        charge_time.insert(0, time)
        
        # Update the remaining capacity
        remains = travel_distance - charge_distance + remains
    
    # The time spent charging at the last station is always 0
    charge_time.append(0)

    return charge_time

#example input
x = [0, 100, 300, 600, 800, 1000]
c = [0, 10, 0, 20, 10, 0]
g = [0.05, 0.2, 0.1, 0.2, 0.1, 0]
print(timeSpent(x, c, g))  # Output: [20.0, 0, 30.0, 40.0, 30.0, 0]

[20.0, 0, 30.0, 40.0, 30.0, 0]


# Task 4

In [1]:
import heapq

def has_cycle(edges):
    visited = set()  # Set to keep track of visited nodes
    graph = {}  # Dictionary to represent the graph structure
    
    # Construct undirected graph
    for edge in edges:  
        u, v = edge  
        graph.setdefault(u, []).append(v)  
        graph.setdefault(v, []).append(u)  

    # Depth First Search (DFS) to detect cycles
    def dfs(node, parent):
        visited.add(node)  # Mark current node as visited
        for neighbor in graph.get(node, []):  # Iterate over neighbors of the current node
            if neighbor == parent:  # Skip the parent node
                continue  
            if neighbor in visited:  # If neighbor is visited and not the parent, there's a cycle
                return True  
            if dfs(neighbor, node):  # Recursively call DFS for unvisited neighbors
                return True
        return False  

    # Main function to check if the graph has a cycle
    for node in graph:
        if node not in visited:  # Start DFS from unvisited nodes  
            if dfs(node, None):  # If cycle detected, return True
                return True  
    return False  # Return False if no cycle is found

def modifiedMST(graph: dict):
    start_node = list(graph.keys())[0]  # Choose the first node as the starting point
    visited = {start_node}  # Set to keep track of visited nodes during traversal
    min_spanning_tree = {}  # Dictionary to store the minimum spanning tree
    edges = [(cost, start_node, neighbor) for neighbor, cost in graph[start_node].items()]  # Create initial edge list
    heapq.heapify(edges)  # Convert the edge list into a min-heap

    # Prim's algorithm for finding the minimum spanning tree
    while edges:
        cost, node1, node2 = heapq.heappop(edges)  # Pop the edge with the minimum cost
        if node2 not in visited:  # If the destination node is not visited
            visited.add(node2)  # Mark it as visited
            # Add the edge to the minimum spanning tree
            if node1 not in min_spanning_tree:
                min_spanning_tree[node1] = {}
            min_spanning_tree[node1][node2] = cost  
            if node2 not in min_spanning_tree:
                min_spanning_tree[node2] = {}
            min_spanning_tree[node2][node1] = cost  
            # Explore neighbors of the destination node
            for neighbor, cost in graph[node2].items():  
                if neighbor not in visited:  
                    heapq.heappush(edges, (cost, node2, neighbor))  

    mst_edges = []  # List to store edges of the minimum spanning tree
    for node in min_spanning_tree:
        for neighbor, weight in min_spanning_tree[node].items():
            if (neighbor, node) not in mst_edges:
                mst_edges.append((node, neighbor))

    remaining_graph = {}  # Dictionary to store the remaining edges after constructing the MST
    for node in graph:
        for neighbor in graph[node]:
            if neighbor not in min_spanning_tree[node]:
                if node not in remaining_graph:
                    remaining_graph[node] = {}
                remaining_graph[node][neighbor] = graph[node][neighbor]

    back_edges = {}  # Dictionary to store back edges added to the MST
    for i in range(len(mst_edges)):
        mst_edge = mst_edges[i]  # Current edge being considered
        src, dest = mst_edge
        edges_graph = {}

        # Determine the graph structure considering the current edge
        if src not in remaining_graph:
            src_value = {}
        else:
            src_value = remaining_graph[src]
        if dest not in remaining_graph:
            dest_value = {}
        else:
            dest_value = remaining_graph[dest]

        edges_graph[src] = src_value
        edges_graph[dest] = dest_value
        # Sort edges by weight for each node in the graph
        sorted_edges = sorted([(node, neighbor, weight) for node in edges_graph for neighbor, weight in edges_graph[node].items()],
                              key=lambda x: x[2])

        for edge in sorted_edges:
            value = (edge[0], edge[1])
            mst_edges[i] = value

            # Check if adding the current edge creates a cycle
            if(not has_cycle(mst_edges)):
                back_edges[mst_edge] = value
                break
        mst_edges[i] = mst_edge  # Restore the current edge

    return back_edges  # Return the dictionary of back edges

# Example input
graph = {'A': {'B': 4, 'C': 8},
         'B': {'A': 4, 'C': 2, 'D': 3, 'E': 5},
         'C': {'A': 8, 'B': 2, 'D': 1, 'E': 6},
         'D': {'B': 3, 'C': 1, 'E': 7},
         'E': {'B': 5, 'C': 6, 'D': 7}}

print(modifiedMST(graph))  # Output: {('A', 'B'): ('A', 'C'), ('B', 'C'): ('B', 'D'), ('B', 'E'): ('E', 'C'), ('C', 'D'): ('D', 'B')}


{('A', 'B'): ('A', 'C'), ('B', 'C'): ('B', 'D'), ('B', 'E'): ('E', 'C'), ('C', 'D'): ('D', 'B')}


# Task 5

## T5-1

### Recurrence Relation

The recurrence relation for the running time of the algorithm is:

$ T(n) = T(n/3) + O(1) $

where $ T(n) $ is the running time of the algorithm on an input of size $ n $, and $ O(1) $ represents the constant time to perform comparisons and sum operations.

### Solution Methods

#### Master Theorem:

The recurrence fits the form $ T(n) = aT(\frac{n}{b}) + f(n) $ where $ a = 1 $, $ b = 3 $, and $ f(n) = O(1) $.

According to the Master Theorem, since $ f(n) = O(n^c) $ where $ c = 0 $ and $ \log_b{a} = \log_3{1} = 0 $, the running time is $ T(n) = \Theta(n^{\log_b{a}}) = \Theta(\log{n}) $.

#### Recursion Tree Method:

At each step, the problem is divided into three equal parts. We can represent this as a recursion tree. The tree has a height of $ \log_3{n} $ since the problem size decreases by a factor of 3 at each level. At each level, there is a constant-time operation $O(1)$.

$
T(n) = O(1) + O(1) + \ldots + O(1) =O(\log n)
$

$
T(n) = O（logn）
$

Therefore, the total running time is $ O(\log{n}) $.


## T5-2

### Running Time

The running time of the `heapify` operation in a d-ary heap is:

$ T(n) = O(d \log_d n) $

where $ T(n) $ is the running time of the `heapify` operation on a heap of size $ n $, and $ d $ is the degree of the heap.

The `insert` operation in a d-ary heap has a running time of:

$ T(n) = O(\log_d n) $

where $ T(n) $ is the running time of the `insert` operation on a heap of size $ n $, and $ d $ is the degree of the heap.

### Comparison with Binary Heap

In terms of theoretical time complexity, both d-ary heap sort and binary heap sort have a time complexity of:

$ T(n) = O(n \log n) $

where $ T(n) $ is the running time of the sorting algorithm on an input of size $ n $. This is because both algorithms involve building a heap (which takes $ O(n) $ time) and then repeatedly removing the maximum element and heapifying the remaining elements (which takes $ O(\log n) $ time per operation, repeated $ n $ times).

However, the base of the logarithm in the time complexity is different (it's $ d $ for d-ary heaps and $ 2 $ for binary heaps), which can lead to different actual runtimes. A d-ary heap can potentially be faster than a binary heap for heap sizes that exceed the size of the computer's cache memory.


## T5-3

### Design:

The algorithm uses two main functions: `calculate_charge_time` and `timeSpent`.

1. `calculate_charge_time`: This function calculates the total time spent charging at a station. It takes into account the wait time at the station and the time it takes to charge the car based on the charging speed at the station.

2. `timeSpent`: This function uses dynamic programming to find the minimum time spent charging at each station. It initializes a memoization table `min_time` to store the minimum time to reach each station for each possible remaining capacity of the car. It also maintains a table `last_charge_distance` to store the last charge distance that leads to the minimum time at each station.

### Correctness:

The correctness of the algorithm can be shown by induction.

- Base case: For the first station, the minimum time is 0 as the car starts at this station.

- Inductive step: Assume the algorithm works for the first $ i$ stations. For the $i+1$th station, the algorithm calculates the minimum time by considering all possible charge distances at the $i$th station that would allow the car to reach the $i+1$th station. It updates the minimum time and the corresponding charge distance for the $i+1$th station. Therefore, the algorithm works for the $i+1$th station.

### Time Complexity Calculation:

1. **`calculate_charge_time` Function**:
   - The time complexity of this function is constant because it performs simple mathematical operations and does not involve loops or recursion. Therefore, the time complexity is O(1).

2. **`timeSpent` Function**:
   - **Forward Pass**:
     - During the forward pass, we iterate over each station and perform a series of calculations for each station, including looping to calculate remaining capacity and charge distance. The innermost loop's iterations depend on the remaining capacity and charge distance, with a worst-case complexity of O(𝑚^2), where 𝑚 is the car's capacity. Therefore, the time complexity of the forward pass is O(𝑛 * 𝑚^2), where 𝑛 is the number of stations.
   - **Backtracking**:
     - Backtracking only requires traversing each station once, so the time complexity is O(𝑛).

Therefore, the overall time complexity is the sum of the time complexities of the forward pass and backtracking, which simplifies to O(𝑛 * 𝑚^2) + O(𝑛), which is O(𝑛 * 𝑚^2).

### Space Complexity Calculation:

1. **`calculate_charge_time` Function**:
   - The space complexity of this function is constant because it only uses a few local variables, which do not increase with input size. Therefore, the space complexity is O(1).

2. **`timeSpent` Function**:
   - We use two 2D arrays of size (𝑛 * 𝑚) to store `min_time` and `last_charge_distance`. Therefore, the total space complexity of these two arrays is O(𝑛 * 𝑚).
   - Additionally, we use some additional local variables such as remaining capacity and charge time, but their space usage is constant and can be ignored.

Therefore, the overall space complexity is dominated by the `min_time` and `last_charge_distance` arrays, making it $ O(𝑛 * 𝑚) $.





## T5-4

$NO$

Given the graph G from Task 4, it is not possible to have more than one minimum spanning tree (MST). Here’s the justification:

In the construction of an MST, we typically select the edge with the smallest weight that does not form a cycle with the already selected edges. When all edge weights are unique, as in the graph G , there is only one choice in the selection of the next edge. This means that there is only one way to construct the MST. This is true for both Kruskal’s and Prim’s algorithms, which are commonly used for constructing MSTs.

Furthermore, a backup edge in the context of MSTs is an edge that is not included in the MST but could replace an edge in the MST without increasing the total weight and without forming a cycle. If there are edges in the graph with the same weight, then there may be backup edges, leading to the possibility of multiple distinct MSTs. However, in the graph G, all edge weights are unique, so there are no backup edges. Therefore, there is only one MST.

In conclusion, the uniqueness of edge weights in the graph leads to a single MST because it results in a single choice at each step of the MST construction and the absence of backup edges with equal weights.